# Measuring the spin exchange rate

In [1]:
#import statements
import numpy as np
import matplotlib.pyplot as plt
import one_ring.base.fitting_tools as fitt
import one_ring.daq.nidaqmx_tools as ndaq
import one_ring.daq.stanford_research_systems as srs
import importlib
import one_ring.base.signal_tools as sit
from one_ring.base.pint_units import ureg
import one_ring.base.h5py_tools as h5t
import scipy.constants
from scipy import constants
%matplotlib widget
import time as tm
import pyvisa as visa
import one_ring.daq.visa_instruments as nvisa
plt.style.use("dark_background")
import pandas as pd
import one_ring.daq.thorlabs as thor
import one_ring.daq.tektronix as tek
import one_ring.base.pandas_tools as pdt
import one_ring.base.hdf5_tools as hdf

# Magnetic Field

In [22]:
larmor(0.64*ureg.uT)

<Quantity(4.48399251, 'kilohertz')>

In [3]:
def larmor(B): #if the pump is on
    return (scipy.constants.physical_constants['electron gyromagn. ratio over 2 pi'][0] * ureg.MHz / ureg.T/ 4 * B).to(ureg.kHz)
def larmor2(B):
    return (14*ureg.kHz / ureg.uT * B).to(ureg.kHz)


In [4]:
B_ambient = np.array([22.5, 10.5, 7.0]) * ureg.uT
B_3axis = np.array([0.2, 0.2 , 0.2]) * ureg.uT /ureg.mA
R_3axis = np.array([11.2, 9.5, 8.1]) * ureg.ohm
LDC_calib = (50.63*ureg.mA/ureg.V)

B_target = ([0, 50, 0]) * ureg. uT
B_kick = ([4, 0, 0]) * ureg.nT

In [5]:
#Volts to cancel ambient field
V_cancel = ((B_ambient / B_3axis) * R_3axis).to(ureg.V)
V_final = ((B_target / B_3axis) * R_3axis).to(ureg.V)
V_kick = ((B_kick/B_3axis) * R_3axis).to(ureg.V)

For modulating the Bx field to cause a 1uT kick i need to modulate the Bx field across these currents
112.42 * ureg.mA --> 117.38 * ureg.mA

AO to LDC is a 50.63 mA/V calibration (162.92 -112.28) mA / (1-0 V)

In [6]:
V_cancel + V_final

Magnitude,[1.26 2.8737500000000002 0.28350000000000003]
Units,volt


In [ ]:
V_kick + V_cancel + V_final

In [ ]:
#analogue output voltage to get desired kick
(V_kick / R_3axis).to(ureg.mA) / LDC_calib

In [ ]:
larmor2(np.sqrt(B_target[0]**2 + B_target[1]**2 + B_target[2]**2))

In [ ]:
larmor(np.sqrt(B_target[0]**2 + B_target[1]**2 + B_target[2]**2))

# Spectrum Analyzer

In [ ]:
bw = 482*ureg.kHz
f_res = 1*ureg.Hz
SA = ndaq.SpectrumAnalyzer(bw, f_res)

In [ ]:
SA.configure_channels()

In [ ]:
SA.display()

In [ ]:
(500*ureg.mA * 1*ureg.V/10/ureg.V) * 0.2 * ureg.uT / ureg.mA

In [ ]:
SA.get_data().iloc[210] #at 211 Hz

In [ ]:
SA.get_data().iloc[1100-1] #at 1.1 kHz

In [ ]:
SA.get_dataa().iloc[11000-1] #at 11 kHz

In [ ]:
SA.get_data().iloc[55000-1] #at 55 kHz

In [ ]:
SA.get_data().iloc[110000-1] #at 110 kHz

In [ ]:
SA.get_data().iloc[252000-1] #at 252 kHz

# PDB Viewer

In [10]:
PDB_Monitor.close()

In [11]:
fs = 200 * ureg.Hz #f3dB is 159kHz
dt = 1/fs
PDB_Monitor = ndaq.DAQViewer(dt)

In [12]:
PDB_Monitor.configure_channels()

In [13]:
PDB_Monitor.display()

Output()

# Finite Generation Measurement

In [ ]:
FGM.close()

In [ ]:
fs = 1 * ureg.MHz
acq_time = 1*ureg.s
ao_sample_rate = fs
FGM = ndaq.FiniteGenerationAndMeasurement(fs,acq_time, ao_sample_rate)

In [ ]:
FGM.configure_channels()

In [ ]:
fs_out = FGM.configure_timings()

In [ ]:
samples_per_call = int(round(1*ureg.s*fs_out))
yi = 0
yf =  0.098
transition_time = 10e-10
hold_time = 1e-7*ureg.s
t_start = 0.1 *ureg.s
t2_expected = 100*ureg.us
field = 50*ureg.uT
field_freq = larmor(field)
field_period = (1/field_freq).to(ureg.s)
signal = sit.TanhWindow(fs_out, 2000, yi, yf, transition_time, hold_time,t_start=t_start.m)

In [ ]:
interval = 1
signal_x, signal_y = signal.generate_for(interval)

In [ ]:
FGM.configure_tasks(signal_y)

In [ ]:
FGM.start()

In [ ]:
read_data = FGM.read(blocking=True)

In [ ]:
FGM.stop()

In [ ]:
data = read_data[0,:]
t_array = FGM.ai_task.generate_time(read_data.size)

In [ ]:
fig, axes = plt.subplots(figsize=(14,4),ncols=3)
ax = axes[0]
ax1 = axes[1]
ax2 = axes[2]
#ax.plot(signal_x, signal_y,label='Analogue Out')
ax.plot(t_array.m, data,label='Measured Voltage')
ax1.plot(t_array.m, data,label='Measured Voltage')
ax2.plot(t_array.m, data,label='Measured Voltage')
#ax.plot(signal_x, signal_y,label='Analogue Out')
#ax.plot(signal_x, data,label='Measured Voltage')

ax.set_xlabel('Time [s]')
ax.set_ylabel('Voltage [V]')
for axis in [ax, ax1,ax2]:
    axis.axvline(t_start.m,alpha=0.3)
    axis.axvspan(t_start.m, (t_start+hold_time.to(ureg.s)).m ,alpha=0.1,color='white')
    axis.axvspan(t_start.m, (t_start + t2_expected).to(ureg.s).m,alpha=0.3, color='red')
    axis.axvspan(t_start.m, (t_start + field_period).to(ureg.s).m,alpha=0.3, color='green')
    axis.set_xlabel('Time [s]')

ax.set_title('full data')
ax1.set_title('pump off region')
ax2.set_title('T2 decay, ~100 us')

ax.set_xlim(0.0,1)
ax1.set_xlim(0.09, 0.104)
ax2.set_xlim(t_start.to(ureg.s).m - 0.00003, (t_start + t2_expected).to(ureg.s).m)

plt.tight_layout()
plt.show()

In [ ]:
sdf  = sit.SignalFilterer(t=t,y=y)

In [ ]:
fig, axes = plt.subplots(figsize=(15,5),ncols=2)
ax = axes[0]
ax1 = axes[1]
sdf.plot_spectral_density(ax=ax)
ax1.plot(t,y)
ax.set_xlim(40000, 120000)

In [ ]:
(data.max() - data.min()) * ureg.V

In [ ]:
50 kHz => 1.00 Vpp
100 kHz => 0.981 Vpp
200 kHz => 0.918 Vpp
300 kHz => 0.834 Vpp

100 Hz => 1.006 Vpp
20 kHz => 0.98 Vpp
50 kHz => 0.87 Vpp
100 kHz => 0.66 Vpp
200 kHz => 0.41 Vpp
300 kHz => 0.28 Vpp

100 Hz => 1.005 Vpp
1 kHz => 1.0048 Vpp
10 kHz => 0.95 Vpp
20 kHz => 0.84 Vpp
50 kHz => 0.52 Vpp
100 kHz => 0.30 Vpp
200 kHz => 0.16 Vpp
300 kHz => 0.10 Vpp

In [ ]:
larmor(field)

In [ ]:
df = pd.DataFrame(data,t_array.m, columns=['Voltage'])
end_series = (0.1*ureg.s + (200*ureg.us)).to(ureg.s).m
fid_df = df[(df.index>=0.099999) & (df.index < end_series)]

In [ ]:
fitter = fitt.T2DecayFitter()

In [ ]:
fit =fitter.fit(fid_df.index.to_numpy(), fid_df['Voltage'].to_numpy(),guess=(0.08,350000,0,0,1,41000))

In [ ]:
fig, ax = plt.subplots()
ax.plot(fid_df.index, fid_df,label='AO')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Voltage [V]')
ax.legend()
fit.plot_fit(fkwargs={'alpha':0.8})
plt.show()

In [ ]:
(1/((0.1000526 - 0.10002821) * ureg.s)).to(ureg.Hz)

In [ ]:
fit.fit_parameters

In [ ]:
(1/larmor(10*ureg.uT)).to(ureg.s)

In [ ]:
((154.60-149.55) * ureg.mA) / (0.1 * ureg.V)

In [ ]:
5 * ureg.uT / (0.2*ureg.uT/ureg.mA)